In [1]:
import os
import re
import math
from random import random, randint, choice
from functools import lru_cache, partial
import yaml

In [2]:
WORKDIR = '.'
CONFIG_FILE = os.path.join(WORKDIR, 'exp3_default_config.yml')
TEMP_CONFIG_FILE = os.path.join(WORKDIR, 'exp3_temp_config.yml')
SIMULATOR = os.path.join(WORKDIR, '../build/simulator')
TRACE_DIR = os.path.join(WORKDIR, '../traces')
TRACES = os.listdir(TRACE_DIR)

CACTI_DIR = os.path.join(WORKDIR, '../../../lab3/cacti65')
CACTI = os.path.join(CACTI_DIR, 'obj_opt/cacti')
CACHE_CFG = os.path.join(WORKDIR, 'cache.cfg')
TEMP_CACHE_CFG = os.path.join(WORKDIR, 'cache_temp.cfg')

SIMULATOR_FREQUENCY = 2  # GHz


In [3]:
re_access_time = re.compile(r'Access time \(ns\): (\d+\.\d+)')

@lru_cache(maxsize=None)
def get_cache_hit_cycle(size, associativity, cache_line_bytes):
    with open(TEMP_CACHE_CFG, 'w') as f:
        f.write('-size (bytes) {}\n'.format(size * 1024))
        f.write('-block size (bytes) {}\n'.format(cache_line_bytes))
        f.write('-associativity {}\n'.format(associativity))
        with open(CACHE_CFG, 'r') as ff:
            f.write(ff.read())
    
    with os.popen('qemu-i386 {} -infile {}'.format(CACTI, TEMP_CACHE_CFG)) as f:
        output = f.read()
        try:
            match = re_access_time.search(output)
            return math.ceil(float(match.group(1)) * SIMULATOR_FREQUENCY)
        except:
            return None

In [4]:
with open(CONFIG_FILE, 'r') as f:
    default_config = yaml.load(f.read())

re_amat = re.compile(r'AMAT: (\d+\.\d+) cycles')

def get_ATMT(trace, cache_config):
    for i in range(len(cache_config)):
        hit_cycle = get_cache_hit_cycle(**cache_config[i])
        if hit_cycle is None:
            return math.inf
        default_config['cache'][i].update(cache_config[i])
        default_config['cache'][i].update({'hit_cycles': hit_cycle})

    with open(TEMP_CONFIG_FILE, 'w') as f:
        f.write(yaml.dump(default_config))
    
    trace = os.path.join(TRACE_DIR, trace)
    with os.popen('{} -c {} {}'.format(SIMULATOR, TEMP_CONFIG_FILE, trace)) as f:
        output = f.read()
        try:
            match = re_amat.search(output)
            return float(match.group(1))
        except:
            return math.inf


In [27]:
def SA(init, gen_func, cost_func, copy_func):
    S = (init, cost_func(init))
    best = (copy_func(S[0]), S[1])
    print(best)
    T = 8e-1
    alpha = 0.9
    while T > 1e-4:
        temp = gen_func(S[0])
        Sp = (temp, cost_func(temp))
        print('Temp:', T)
        print(Sp)
        if Sp[1] < S[1]:
            S = Sp
            if S[1] < best[1]:
                best = (copy_func(S[0]), S[1])
                print('!!! best:', best)
        else:
            if random() < math.exp((S[1] - Sp[1]) / T):
                S = Sp
        T *= alpha
    return best

In [34]:
ATTRS = ['size', 'associativity', 'cache_line_bytes']
ATTR_MIN_MAX = {
    'size': (32, 32768),
    'associativity': (2, 16),
    'cache_line_bytes': (256, 8192),
}
# cache_config = [{k: cc[k] for k in cc if k in ATTRS} for cc in default_config['cache']]
cache_config = [{'size': 128, 'associativity': 2, 'cache_line_bytes': 4096}, {'size': 512, 'associativity': 4, 'cache_line_bytes': 8192}, {'size': 8192, 'associativity': 4, 'cache_line_bytes': 8192}]

def copy_config(cache_config):
    return [cc.copy() for cc in cache_config]

def gen_new_config(cache_config):
    new_config = copy_config(cache_config)
    level = choice([0, 1, 2, 2, 2])
    attr = choice(ATTRS)
    if (randint(0, 3) <= 1 and new_config[level][attr] > ATTR_MIN_MAX[attr][0]) or new_config[level][attr] >= ATTR_MIN_MAX[attr][1]:
        new_config[level][attr] //= 2
    else:
        new_config[level][attr] *= 2
    # print(new_config)
    return new_config

trace = TRACES[0]
# print((cache_config, get_ATMT(trace, cache_config)))
best_config = SA(cache_config, gen_new_config, partial(get_ATMT, trace), copy_config)

print('========================= best of all ================================')
print((best_config[0], get_ATMT(trace, best_config[0])))

([{'size': 128, 'associativity': 2, 'cache_line_bytes': 4096}, {'size': 512, 'associativity': 4, 'cache_line_bytes': 8192}, {'size': 8192, 'associativity': 4, 'cache_line_bytes': 8192}], 1.11)
Temp: 0.8
([{'size': 128, 'associativity': 2, 'cache_line_bytes': 4096}, {'size': 512, 'associativity': 4, 'cache_line_bytes': 8192}, {'size': 8192, 'associativity': 2, 'cache_line_bytes': 8192}], 1.11)
Temp: 0.7200000000000001
([{'size': 128, 'associativity': 2, 'cache_line_bytes': 4096}, {'size': 512, 'associativity': 4, 'cache_line_bytes': 4096}, {'size': 8192, 'associativity': 2, 'cache_line_bytes': 8192}], 1.12)
Temp: 0.6480000000000001
([{'size': 128, 'associativity': 2, 'cache_line_bytes': 4096}, {'size': 512, 'associativity': 4, 'cache_line_bytes': 4096}, {'size': 8192, 'associativity': 4, 'cache_line_bytes': 8192}], 1.12)
Temp: 0.5832000000000002
([{'size': 128, 'associativity': 2, 'cache_line_bytes': 4096}, {'size': 512, 'associativity': 4, 'cache_line_bytes': 4096}, {'size': 8192, 'ass

KeyboardInterrupt: 

In [25]:

# print((cache_config, get_ATMT(trace, cache_config)))

([{'size': 128, 'associativity': 2, 'cache_line_bytes': 4096}, {'size': 512, 'associativity': 4, 'cache_line_bytes': 8192}, {'size': 8192, 'associativity': 4, 'cache_line_bytes': 8192}], 1.11)
